# `xWRF` and `xgcm`

In this tutorial, we will show you how to leverage the `xWRF`-provided [COMODO](https://web.archive.org/web/20160417032300/http://pycomodo.forge.imag.fr/norm.html)-compliant attributes with `xgcm` in order to destagger the WRF output.

## Loading the data

First of all, we load the data and use the simple `.xwrf.postprocess()` API.

In [1]:
import xwrf

ds = xwrf.tutorial.open_dataset("wrfout").xwrf.postprocess()
ds

<xarray.Dataset>
Dimensions:                    (y: 185, x: 200, z: 41, x_stag: 201, y_stag: 186)
Coordinates:
    XLAT                       (y, x) float32 ...
    XLONG                      (y, x) float32 ...
    XTIME                      datetime64[ns] ...
  * y                          (y) float64 -9.949e+04 -9.849e+04 ... 8.451e+04
  * y_stag                     (y_stag) float64 -9.999e+04 ... 8.501e+04
  * x_stag                     (x_stag) float64 -8.844e+04 ... 1.116e+05
  * x                          (x) float64 -8.794e+04 -8.694e+04 ... 1.111e+05
  * z                          (z) float32 0.9969 0.9899 ... 0.008311 0.002679
    Time                       datetime64[ns] 2018-10-16
Data variables:
    Times                      |S19 b'2018-10-16_00:00:00'
    U                          (z, y, x_stag) float32 ...
    V                          (z, y_stag, x) float32 ...
    QVAPOR                     (z, y, x) float32 ...
    PSN                        (y, x) float32 ...
    wrf_projection             object +proj=lcc +x_0=0 +y_0=0 +a=6370000 +b=6...
    air_potential_temperature  (z, y, x) float32 293.1 293.0 ... 496.7 496.7
    air_pressure               (z, y, x) float32 9.606e+04 ... 5.255e+03
Attributes: (12/149)
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      2018-10-15_00:00:00
    SIMULATION_START_DATE:           2018-10-15_00:00:00
    WEST-EAST_GRID_DIMENSION:        201
    SOUTH-NORTH_GRID_DIMENSION:      186
    BOTTOM-TOP_GRID_DIMENSION:       42
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2

If we naively try to calculate the wind speed from the `U` and `V` components, we get an error due to them having different shapes.

In [2]:
from metpy.calc import wind_speed

wind_speed(ds.U, ds.V)

ValueError: operands could not be broadcast together with shapes (41,185,201) (41,186,200) 

Upon investigating the wind components (here `U`), we can see that they are defined on the WRF-internal Arakawa-C grid, which causes the shapes to differ.

In [3]:
ds.U

<xarray.DataArray 'U' (z: 41, y: 185, x_stag: 201)>
array([[[-0.292543,  0.084697, ...,  2.125366,  2.215864],
        [-0.315306,  0.035766, ...,  1.970581,  2.155364],
        ...,
        [ 3.244892,  3.038538, ...,  0.81328 ,  0.607562],
        [ 3.224946,  3.064827, ...,  1.017266,  0.825519]],

       [[-3.009571, -2.741389, ...,  4.688902,  4.777993],
        [-2.99956 , -2.736013, ...,  5.031726,  4.688388],
        ...,
        [ 1.647537,  1.518185, ..., -1.415898, -1.498976],
        [ 1.738011,  1.51681 , ..., -1.363423, -1.385749]],

       ...,

       [[ 0.90752 ,  0.914122, ..., -0.238839, -0.227455],
        [ 0.915585,  0.895405, ..., -0.118391, -0.218255],
        ...,
        [ 1.837519,  1.818229, ...,  1.019168,  1.013208],
        [ 1.835708,  1.849653, ...,  1.024746,  1.018466]],

       [[-0.567978, -0.575821, ..., -1.001519, -1.024916],
        [-0.558946, -0.605989, ..., -1.097908, -1.014208],
        ...,
        [ 1.423835,  1.409019, ...,  0.191365,  0.268683],
        [ 1.424662,  1.438232, ...,  0.306561,  0.281939]]], dtype=float32)
Coordinates:
    XTIME    datetime64[ns] 2018-10-16
  * y        (y) float64 -9.949e+04 -9.849e+04 ... 8.351e+04 8.451e+04
  * x_stag   (x_stag) float64 -8.844e+04 -8.744e+04 ... 1.106e+05 1.116e+05
  * z        (z) float32 0.9969 0.9899 0.981 ... 0.01452 0.008311 0.002679
    Time     datetime64[ns] 2018-10-16
Attributes:
    FieldType:      104
    MemoryOrder:    XYZ
    description:    x-wind component
    units:          m s-1
    stagger:        X
    standard_name:  x_wind
    grid_mapping:   wrf_projection

But, since `xWRF` prepared the dataset with the appropriate [COMODO](https://web.archive.org/web/20160417032300/http://pycomodo.forge.imag.fr/norm.html) (and `units`) attributes, we can simply use [`xgcm`](https://xgcm.readthedocs.io/en/latest/grids.html) to solve this problem!

In [4]:
from xgcm import Grid


def interp_and_keep_attrs(grid, da, axis):
    _attrs = da.attrs
    da = grid.interp(da, axis=axis)
    _attrs['stagger'] = ''
    da.attrs = _attrs
    return da


grid = Grid(ds)
ds['U'], ds['V'] = interp_and_keep_attrs(grid, ds.U, 'X'), interp_and_keep_attrs(grid, ds.V, 'Y')
wind_speed(ds.U, ds.V)

<xarray.DataArray (z: 41, y: 185, x: 200)>
<Quantity([[[2.5830894  2.4726613  2.3651724  ... 3.8345053  3.7921946  3.562908  ]
  [2.5936317  2.4906101  2.378739   ... 3.7439888  3.6586847  3.3785107 ]
  [2.58631    2.4990115  2.4210165  ... 3.357024   3.2736325  3.1454208 ]
  ...
  [3.2211604  3.0455003  2.8412445  ... 2.1406295  2.097297   2.044071  ]
  [3.2306287  3.0598316  2.8453405  ... 2.1926136  2.1727755  2.1062996 ]
  [3.2863615  3.095959   2.8095798  ... 2.2487814  2.2399218  2.1625416 ]]

 [[2.9969168  2.7385101  2.4665911  ... 7.967784   7.6699953  7.8899965 ]
  [2.9768007  2.713516   2.4515765  ... 7.963011   7.6715903  7.991155  ]
  [2.9442165  2.6654217  2.3883235  ... 7.8314524  7.7466373  7.901966  ]
  ...
  [1.830133   1.7440214  1.7133143  ... 2.1559918  2.3387313  2.5133698 ]
  [1.8968419  1.7740413  1.6545867  ... 1.9062783  2.1123106  2.3065217 ]
  [1.8831187  1.705373   1.5765473  ... 1.646673   1.8946757  2.1010668 ]]

 [[2.3906026  2.244915   2.1361825  ... 5.800373   6.030613   6.0805445 ]
  [2.4904559  2.337277   2.198147   ... 5.6592727  5.8733068  5.917687  ]
  [2.5388439  2.4170237  2.2534733  ... 5.6435056  5.7729797  5.8758607 ]
  ...
...
  ...
  [4.0241203  4.0401917  4.0316167  ... 4.5777645  4.666469   4.7366095 ]
  [4.024752   4.050392   4.0493803  ... 4.516429   4.6022058  4.714714  ]
  [4.019227   4.030483   4.0321116  ... 4.5582085  4.6036725  4.684994  ]]

 [[4.657378   4.66478    4.6701903  ... 6.2175217  6.192114   6.1512747 ]
  [4.6613345  4.6758523  4.688787   ... 6.2607093  6.2101817  6.143415  ]
  [4.666457   4.6776147  4.689986   ... 6.2288     6.1813846  6.1367726 ]
  ...
  [3.7959666  3.7874937  3.7947266  ... 4.5482545  4.5661564  4.5791197 ]
  [3.7967908  3.7786262  3.781312   ... 4.5328298  4.552653   4.5703735 ]
  [3.8021736  3.7941277  3.7946591  ... 4.5263104  4.5440474  4.561946  ]]

 [[7.2251277  7.263054   7.2985206  ... 7.785804   7.808089   7.8435926 ]
  [7.2128506  7.265067   7.315435   ... 7.756877   7.7991524  7.8451815 ]
  [7.19954    7.2502933  7.305122   ... 7.78237    7.820182   7.841373  ]
  ...
  [3.600367   3.5728045  3.5593214  ... 7.02204    6.982752   6.9361553 ]
  [3.603963   3.5700598  3.5528288  ... 7.0524116  7.0145397  6.9407434 ]
  [3.6098907  3.5873144  3.5720725  ... 6.9871693  6.9808655  6.946658  ]]], 'meter / second')>
Coordinates:
    XLAT     (y, x) float32 48.66 48.66 48.66 48.66 ... 50.31 50.31 50.31 50.31
    XLONG    (y, x) float32 7.302 7.316 7.329 7.343 ... 10.02 10.04 10.05 10.07
    XTIME    datetime64[ns] 2018-10-16
  * y        (y) float64 -9.949e+04 -9.849e+04 ... 8.351e+04 8.451e+04
  * x        (x) float64 -8.794e+04 -8.694e+04 ... 1.101e+05 1.111e+05
  * z        (z) float32 0.9969 0.9899 0.981 ... 0.01452 0.008311 0.002679
    Time     datetime64[ns] 2018-10-16